In [1]:
import tensorflow as tf
import os
import utils
import word2vec_utils
import numpy as np

In [42]:
BATCH_SIZE = 128
VOCAB_SIZE = 50000
SKIP_WINDOW = 1 
VISUAL_FLD = 'visualization'
EMBED_SIZE = 128            # dimension of the word embedding vectors

DOWNLOAD_URL = 'http://mattmahoney.net/dc/text8.zip'
EXPECTED_BYTES = 31344016
NUM_VISUALIZE = 3000        # number of tokens to visualize


In [33]:
def gen():
    yield from word2vec_utils.batch_gen(DOWNLOAD_URL, EXPECTED_BYTES, VOCAB_SIZE, 
                                        BATCH_SIZE, SKIP_WINDOW, VISUAL_FLD)

In [18]:
type([('UNK', -1)])

list

In [34]:
dataset = tf.data.Dataset.from_generator(gen, 
                                (tf.int32, tf.int32), 
                                (tf.TensorShape([BATCH_SIZE]), tf.TensorShape([BATCH_SIZE, 1])))

In [35]:
iterator = dataset.make_initializable_iterator()
center_words, target_words = iterator.get_next()

In [69]:
index_words[0:15]

[5234, 3081, 12, 6, 195, 2, 3134, 46, 59, 156, 128, 742, 477, 10572, 134]

In [36]:
with tf.Session() as sess:
    sess.run(iterator.initializer)   
    t1=sess.run([center_words, target_words])
    print(t1)

data/text8.zip already exists
[array([ 5234,  3081,  3081,    12,    12,     6,     6,   195,   195,
           2,     2,  3134,  3134,    46,    46,    59,    59,   156,
         156,   128,   128,   742,   742,   477,   477, 10572, 10572,
         134,   134,     1,     1, 27350, 27350,     2,     2,     1,
           1,   103,   103,   855,   855,     3,     3,     1,     1,
       15068, 15068,     0,     0,     2,     2,     1,     1,   151,
         151,   855,   855,  3581,  3581,     1,     1,   195,   195,
          11,    11,   191,   191,    59,    59,     5,     5,     6,
           6, 10713, 10713,   215,   215,     7,     7,  1325,  1325,
         105,   105,   455,   455,    20,    20,    59,    59,  2732,
        2732,   363,   363,     7,     7,  3673,  3673,     1,     1,
         709,   709,     2,     2,   372,   372,    27,    27,    41,
          41,    37,    37,    54,    54,   540,   540,    98,    98,
          12,    12,     6,     6,  1424,  1424,  2758,  27

In [41]:
print(type(t1[0]))
print(t1[0])
cw=t1[0]
print(cw.size)

<class 'numpy.ndarray'>
[ 5234  3081  3081    12    12     6     6   195   195     2     2  3134
  3134    46    46    59    59   156   156   128   128   742   742   477
   477 10572 10572   134   134     1     1 27350 27350     2     2     1
     1   103   103   855   855     3     3     1     1 15068 15068     0
     0     2     2     1     1   151   151   855   855  3581  3581     1
     1   195   195    11    11   191   191    59    59     5     5     6
     6 10713 10713   215   215     7     7  1325  1325   105   105   455
   455    20    20    59    59  2732  2732   363   363     7     7  3673
  3673     1     1   709   709     2     2   372   372    27    27    41
    41    37    37    54    54   540   540    98    98    12    12     6
     6  1424  1424  2758  2758    19    19   568]
128


In [44]:
matrix = tf.get_variable('embed_matrix',
                                        shape=[VOCAB_SIZE, EMBED_SIZE],
                                        initializer=tf.random_uniform_initializer())
embed = tf.nn.embedding_lookup(matrix, cw, name='embedding')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    t2=sess.run([matrix, embed])
    print(t2)

[array([[0.5857949 , 0.39738178, 0.316985  , ..., 0.7479669 , 0.35447657,
        0.5607126 ],
       [0.8879    , 0.06618071, 0.07745922, ..., 0.665746  , 0.48201287,
        0.7440702 ],
       [0.7022183 , 0.28129148, 0.6190965 , ..., 0.473724  , 0.4520824 ,
        0.72576654],
       ...,
       [0.8196589 , 0.24916649, 0.58480513, ..., 0.89530635, 0.7686573 ,
        0.21347094],
       [0.676559  , 0.86022353, 0.48458982, ..., 0.7456356 , 0.59780717,
        0.04529643],
       [0.3679073 , 0.03261995, 0.39265168, ..., 0.11875618, 0.66920555,
        0.45949137]], dtype=float32), array([[0.2895224 , 0.8075273 , 0.6718035 , ..., 0.12125194, 0.780877  ,
        0.13041902],
       [0.5692445 , 0.9743459 , 0.84972167, ..., 0.3840276 , 0.6603266 ,
        0.1316632 ],
       [0.5692445 , 0.9743459 , 0.84972167, ..., 0.3840276 , 0.6603266 ,
        0.1316632 ],
       ...,
       [0.06899464, 0.46470976, 0.91744256, ..., 0.4299885 , 0.12206829,
        0.42168903],
       [0.06899464

In [48]:
t2[0].shape

(50000, 128)

In [55]:
t2[0][5234]

array([0.2895224 , 0.8075273 , 0.6718035 , 0.60945   , 0.44296467,
       0.7561674 , 0.42901242, 0.2659247 , 0.12689221, 0.6585249 ,
       0.21787298, 0.5546049 , 0.05459678, 0.46345985, 0.58838785,
       0.803673  , 0.9992738 , 0.62273324, 0.7765964 , 0.44869983,
       0.73483706, 0.75568223, 0.1165688 , 0.4011314 , 0.89357626,
       0.6707295 , 0.15336943, 0.07634044, 0.9564272 , 0.754912  ,
       0.40156317, 0.79751194, 0.84552276, 0.63211894, 0.9407041 ,
       0.6886319 , 0.55444646, 0.11448371, 0.41511536, 0.9803313 ,
       0.63140833, 0.5232551 , 0.914178  , 0.6564342 , 0.32039058,
       0.30430794, 0.30243134, 0.6423913 , 0.583194  , 0.03740895,
       0.3320248 , 0.39419627, 0.62628603, 0.0452466 , 0.4675933 ,
       0.98596036, 0.32360983, 0.4048941 , 0.05540204, 0.41336238,
       0.11265695, 0.9634161 , 0.7474942 , 0.9315833 , 0.45512235,
       0.71983945, 0.33835363, 0.903579  , 0.6039262 , 0.37284696,
       0.38838315, 0.57908356, 0.44614422, 0.03583515, 0.03626

In [58]:
t2[1]

array([[0.2895224 , 0.8075273 , 0.6718035 , ..., 0.12125194, 0.780877  ,
        0.13041902],
       [0.5692445 , 0.9743459 , 0.84972167, ..., 0.3840276 , 0.6603266 ,
        0.1316632 ],
       [0.5692445 , 0.9743459 , 0.84972167, ..., 0.3840276 , 0.6603266 ,
        0.1316632 ],
       ...,
       [0.06899464, 0.46470976, 0.91744256, ..., 0.4299885 , 0.12206829,
        0.42168903],
       [0.06899464, 0.46470976, 0.91744256, ..., 0.4299885 , 0.12206829,
        0.42168903],
       [0.8344841 , 0.55627453, 0.8004556 , ..., 0.4776492 , 0.78020763,
        0.78412235]], dtype=float32)

In [56]:
t2[0][3081]

array([0.5692445 , 0.9743459 , 0.84972167, 0.82206964, 0.6495608 ,
       0.40676558, 0.20357656, 0.9944018 , 0.29481363, 0.88150895,
       0.71226037, 0.23674381, 0.9166478 , 0.275815  , 0.8601899 ,
       0.7308221 , 0.4279498 , 0.3352276 , 0.48107898, 0.2465266 ,
       0.86219513, 0.13595045, 0.8949907 , 0.90325975, 0.35435915,
       0.8926015 , 0.6031252 , 0.69958127, 0.84553885, 0.58696616,
       0.5836046 , 0.9273795 , 0.80726814, 0.06600451, 0.8803251 ,
       0.56940293, 0.08036399, 0.39630675, 0.9369793 , 0.9643992 ,
       0.33129084, 0.77852356, 0.2033658 , 0.42007113, 0.9959024 ,
       0.15478241, 0.30435765, 0.0329361 , 0.02492571, 0.41957188,
       0.43303943, 0.21004367, 0.4250661 , 0.41326082, 0.9541336 ,
       0.04867554, 0.35299754, 0.17306316, 0.03299594, 0.01911831,
       0.22902095, 0.31636047, 0.45968366, 0.37746346, 0.68901694,
       0.8428538 , 0.310004  , 0.5848739 , 0.68565667, 0.6073489 ,
       0.9709977 , 0.5715388 , 0.6158376 , 0.1366936 , 0.76460

In [ ]:
t2[]

In [3]:
import zipfile
def read_data(file_path):
    """ Read data into a list of tokens 
    There should be 17,005,207 tokens
    """
    with zipfile.ZipFile(file_path) as f:
        words = tf.compat.as_str(f.read(f.namelist()[0])).split()
    return words

In [4]:
local_dest = 'data/text8.zip'
utils.download_one_file(DOWNLOAD_URL, local_dest, EXPECTED_BYTES)
words = read_data(local_dest)
print(type(words))

data/text8.zip already exists
<class 'list'>


### There are 17,005,207 tokens if you count tokens by splitting the text by blank space.

In [5]:
words[0]

'anarchism'

In [6]:
words[1]

'originated'

In [7]:
len(words)

17005207

In [10]:
from collections import Counter
utils.safe_mkdir('visualization')
file = open(os.path.join('visualization', 'vocab.tsv'), 'w')
dictionary = dict()
count = [('UNK', -1)]
index = 0
count.extend(Counter(words).most_common(VOCAB_SIZE - 1))
print(type(count))
print(len(count))
print(count[0])
print(count[1])
print(count[2])
print(count[3])
print(count[4])

<class 'list'>
50000
('UNK', -1)
('the', 1061396)
('of', 593677)
('and', 416629)
('one', 411764)


In [11]:
for word, _ in count:
        dictionary[word] = index
       # print(word)
       # print(index)
        index += 1

In [12]:
print(list(dictionary.keys())[0])

UNK


In [13]:
print(list(dictionary.values())[0])

0


In [24]:
print(list(dictionary.keys())[1])

the


In [26]:
print(list(dictionary.values())[1])

1


In [14]:
len(dictionary)

50000

In [15]:
len(words)

17005207

In [16]:
def convert_words_to_index(words, dictionary):
    """ Replace each word in the dataset with its index in the dictionary """
    return [dictionary[word] if word in dictionary else 0 for word in words]

In [17]:
print(words[0],words[1])

anarchism originated


### if 'anarchism' in dictionary ?

In [27]:
'anarchism' in dictionary

True

In [28]:
dictionary['anarchism']

5234

In [29]:
'originated' in dictionary

True

In [30]:
dictionary['originated']

3081

In [31]:
index_words = convert_words_to_index(words, dictionary)

In [32]:
print(index_words[0:2])

[5234, 3081]


In [25]:
 type(index_words)

list

In [50]:
import random
def generate_sample():
    for index,center in enumerate(index_words):
        context = random.randint(1, SKIP_WINDOW)
        print(index,center)
        print(context)
        for target in index_words[max(0, index - context): index]:
            yield center,target

In [57]:
single_gen = generate_sample()

In [58]:
next(single_gen)

0 5234
1
1 3081
1


(3081, 5234)

In [59]:
next(single_gen)

2 12
1


(12, 3081)

In [62]:
next(single_gen)

3 6
1


(6, 12)

In [44]:
def   yield5():
    yield 5
    yield 6

In [46]:
a

<generator object yield5 at 0x7fdce6fb2ba0>

In [47]:
next(a)

5

In [48]:
next(a)

6

In [49]:
next(a)

StopIteration: 

In [51]:
def   yield5():
    yield 5

In [52]:
a=yield5()

In [53]:
next(a)

5

In [54]:
next(a)

StopIteration: 

In [54]:
import random
def generate_sample():
    for index,center in enumerate(index_words):
        context = random.randint(1, SKIP_WINDOW)
        #print(index,center)
        #print(context)
        for target in index_words[max(0, index - context): index]:
            yield center,target
        for target in index_words[index + 1: index + context + 1]:
            yield center, target

In [56]:
single_gen = generate_sample()

In [53]:
index_words[:15]

[5234, 3081, 12, 6, 195, 2, 3134, 46, 59, 156, 128, 742, 477, 10572, 134]

In [36]:
next(single_gen)

0 5234
1


(5234, 3081)

In [37]:
next(single_gen)

1 3081
1


(3081, 5234)

In [38]:
next(single_gen)

(3081, 12)

In [39]:
next(single_gen)

2 12
1


(12, 3081)

In [71]:
next(single_gen)

(12, 6)

In [30]:
a=[0,1,2,3,4,5]

In [32]:
max(0,4)

4

In [33]:
a[3:4]

[3]

In [42]:
center_batch = np.zeros(BATCH_SIZE , dtype=np.int32)

In [43]:
center_batch

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [45]:
target_batch = np.zeros([BATCH_SIZE,1])

In [48]:
target_batch[1]

array([0.])

In [50]:
target_batch[1]=5

In [51]:
target_batch[1]

array([5.])

In [49]:
center_batch[1]

0

In [59]:
for index in range(BATCH_SIZE):
    center_batch[index],target_batch[index]=next(single_gen)
    print(center_batch[index],target_batch[index])

90 [27.]
27 [90.]
27 [271.]
271 [27.]
271 [38.]
38 [271.]
38 [5949.]
5949 [38.]
5949 [4862.]
4862 [5949.]
4862 [20300.]
20300 [4862.]
20300 [29.]
29 [20300.]
29 [0.]
0 [29.]
0 [42.]
42 [0.]
42 [318.]
318 [42.]
318 [6.]
6 [318.]
6 [25637.]
25637 [6.]
25637 [528.]
528 [25637.]
528 [7574.]
7574 [528.]
7574 [372.]
372 [7574.]
372 [5.]
5 [372.]
5 [259.]
259 [5.]
259 [2.]
2 [259.]
2 [154.]
154 [2.]
154 [26.]
26 [154.]
26 [1207.]
1207 [26.]
1207 [12.]
12 [1207.]
12 [7574.]
7574 [12.]
7574 [201.]
201 [7574.]
201 [1577.]
1577 [201.]
1577 [3.]
3 [1577.]
3 [15201.]
15201 [3.]
15201 [333.]
333 [15201.]
333 [1775.]
1775 [333.]
1775 [7089.]
7089 [1775.]
7089 [4861.]
4861 [7089.]
4861 [345.]
345 [4861.]
345 [765.]
765 [345.]
765 [161.]
161 [765.]
161 [407.]
407 [161.]
407 [5691.]
5691 [407.]
5691 [756.]
756 [5691.]
756 [2.]
2 [756.]
2 [4106.]
4106 [2.]
4106 [1132.]
1132 [4106.]
1132 [4332.]
4332 [1132.]
4332 [1537.]
1537 [4332.]
1537 [3.]
3 [1537.]
3 [568.]
568 [3.]
568 [8118.]
8118 [568.]
8118 [99.]

In [61]:
center_batch.size

128